In [1]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans
import folium # map rendering library

In [2]:
post_df_can=pd.read_csv("post_df_can.csv")

In [45]:
post_df_tor=post_df_can[post_df_can.Borough.str.find("Toronto")>0].reset_index().drop(columns=["index"])
post_df_tor

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In [37]:
tor_lat=43.651070
tor_lon=-79.347015

# create map of Manhattan using latitude and longitude values
map_tor = folium.Map(location=[tor_lat, tor_lon], zoom_start=12)

# add markers to map
for lat, lng, label in zip(post_df_tor['Latitude'], post_df_tor['Longitude'], post_df_tor['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

In [47]:
radius=600
LIMIT=100
def getNearbyVenues(names, latitudes, longitudes, radius=600):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
       # print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

**Here I will save this cell and need to define the credentials once and then remove it manually before git.**

In [51]:
CLIENT_ID=""
CLIENT_SECRET=""
VERSION="20180605"

In [48]:
# might take 
toronto_venues = getNearbyVenues(names=post_df_tor['Neighbourhood'],
                                   latitudes=post_df_tor['Latitude'],
                                   longitudes=post_df_tor['Longitude']
                                  )
toronto_venues

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,Tori's Bakeshop,43.672114,-79.290331,Vegetarian / Vegan Restaurant
2,The Beaches,43.676357,-79.293031,The Beech Tree,43.680493,-79.288846,Gastropub
3,The Beaches,43.676357,-79.293031,Beaches Bake Shop,43.680363,-79.289692,Bakery
4,The Beaches,43.676357,-79.293031,Ed's Real Scoop,43.672630,-79.287993,Ice Cream Shop
...,...,...,...,...,...,...,...
2085,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Greenwood Cigar & Variety,43.664538,-79.325379,Smoke Shop
2086,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,ONE Academy,43.662253,-79.326911,Gym / Fitness Center
2087,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Gadabout,43.663788,-79.328229,Thrift / Vintage Store
2088,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,Queen & Coxwell,43.666413,-79.316525,Intersection


In [91]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']
# group the neighbourhood together
toronto_grouped_sum = toronto_onehot.groupby('Neighborhood').sum().reset_index()

In [92]:
toronto_categories=toronto_grouped_sum.set_index('Neighborhood').transpose()
toronto_categories.index.names=["Category"]
cat_top10=toronto_categories.sum(axis = 1).sort_values(ascending=False)[:10].index.to_list()

In [107]:
toronto_grouped=toronto_onehot.groupby("Neighborhood").sum().reset_index()
toronto_grouped=pd.concat([toronto_grouped.loc[:,cat_top10],toronto_grouped["Neighborhood"]],axis=1)

In [149]:
most_cat=[]
most_category=[]
for column in toronto_categories:
    most_cat.append(toronto_categories[column].idxmax())
    most_category.append(": ".join([column,toronto_categories[column].idxmax()]))
most_cat=set(most_cat)

* Here you see the categories appear the most at each neighborhood, so that you get a feeling about each neighborhood, whether it is used for living, transportation, commercial, entertainment or so on.

In [165]:
most_category

['Adelaide, King, Richmond: Coffee Shop',
 'Berczy Park: Coffee Shop',
 'Brockton, Exhibition Place, Parkdale Village: Coffee Shop',
 'Business Reply Mail Processing Centre 969 Eastern: Burrito Place',
 'CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara: Airport Service',
 'Cabbagetown, St. James Town: Coffee Shop',
 'Central Bay Street: Coffee Shop',
 'Chinatown, Grange Park, Kensington Market: Café',
 'Christie: Café',
 'Church and Wellesley: Coffee Shop',
 'Commerce Court, Victoria Hotel: Coffee Shop',
 'Davisville: Pizza Place',
 'Davisville North: Pizza Place',
 'Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West: Coffee Shop',
 'Design Exchange, Toronto Dominion Centre: Coffee Shop',
 'Dovercourt Village, Dufferin: Bakery',
 'First Canadian Place, Underground city: Coffee Shop',
 'Forest Hill North, Forest Hill West: Jewelry Store',
 'Harbord, University of Toronto: Café',
 'Harbourfront East, Toronto Islands

In [150]:
most_cat

{'Airport Service',
 'Bakery',
 'Bar',
 'Bookstore',
 'Burrito Place',
 'Bus Line',
 'Business Service',
 'Café',
 'Coffee Shop',
 'Greek Restaurant',
 'Jewelry Store',
 'Park',
 'Pizza Place',
 'Sandwich Place',
 'Sporting Goods Shop'}

Based on the result above, i would guess number of clusters should be somewhere around 5
- business
- restaurant
- entertainment
- transportation
- others
##### so i will set the k to be 5.

In [163]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped_sum.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)
kmeans.labels_
toronto_grouped_result=pd.merge(toronto_grouped_sum,post_df_tor,how="left",left_on="Neighborhood",right_on="Neighbourhood")
toronto_grouped_result["Cluster"]=kmeans.labels_
toronto_grouped_result=toronto_grouped_result[["Neighborhood","Latitude","Longitude","Cluster"]]
toronto_grouped_result

,Neighborhood,Latitude,Longitude,Cluster
0,"Adelaide, King, Richmond",43.650571,-79.384568,3
1,Berczy Park,43.644771,-79.373306,0
2,"Brockton, Exhibition Place, Parkdale Village",43.636847,-79.428191,2
3,Business Reply Mail Processing Centre 969 Eastern,43.662744,-79.321558,2
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",43.628947,-79.394420,2
5,"Cabbagetown, St. James Town",43.667967,-79.367675,2
6,Central Bay Street,43.657952,-79.387383,1
7,"Chinatown, Grange Park, Kensington Market",43.653206,-79.400049,4
8,Christie,43.669542,-79.422564,2
9,Church and Wellesley,43.665860,-79.383160,1


In [164]:
# create map
map_clusters = folium.Map(location=[tor_lat, tor_lon], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_grouped_result['Latitude'], 
                                  toronto_grouped_result['Longitude'], 
                                  toronto_grouped_result['Neighborhood'], 
                                  toronto_grouped_result['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters